In [167]:
from gensim import corpora, models, similarities
import glob
import os
import logging
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Start by analyzing Q1. We have defined 20 "important" clusters there. Read the tweets for one of these.

In [18]:
userTweets = {}
textPath = "/Users/mikaelhuss1/Desktop/allt/twittercensus/q1"
textFiles = glob.glob(os.path.join(textPath, "*.txt"))
for file in textFiles:
    user = os.path.basename(file).split(".")[0]
    userTweets[user]=open(file).readlines()
userTweets.popitem()[1][:5]

['@stefana77 Hyfsat nära nya appar.\n',
 '@Eko_Invest @gunthermarder @NordnetSE kan hjälpa dig. Kan ha med belåning att göra?\n',
 '@joakimdal @gunthermarder Inom kort kommer nya appar.\n',
 '@mpawlo @gunthermarder Kan du ge mig oddsen?\n',
 '@gunthermarder Vilket spelbolag är först med att sätta odds? @NordicBetSV @unibet\n']

In [26]:
clusterId = {} # dictionary for looking up cluster ID for user ID
userId = {} # dictionary for looking up user IDs for a certain cluster ID
clusterFile = "/Users/mikaelhuss1/Desktop/allt/twittercensus/edges_q1_num_recip_unw.tree.names"
with open(clusterFile) as inFile:
    for line in inFile:
        [fullClusID, score, numIdStr, numId, uName] = line.strip().split()
        twoLevelClusID = ':'.join(fullClusID.split(":")[:2])
        clusterId[uName]=twoLevelClusID
        if not twoLevelClusID in userId:
            userId[twoLevelClusID] = [uName]
        else:
            userId[twoLevelClusID].append(uName)
clusterId.popitem()
userId['1:300']

['Vingresor',
 'pjotorboy',
 'LasseNyberg',
 'KatFilosoferar',
 'beeccisen',
 'flakieflake',
 'LjungMarie',
 'karinericson',
 'Triss66',
 'mariasj80',
 'AnnetteJuslin',
 'Fridajansson',
 'NybergKatta',
 'CSundnas',
 'lalellish',
 'Mitt_Universum',
 'KatalinasMind',
 'karolinagoth',
 'emoyenne',
 'henrikhansense',
 'annebmathisen',
 'NinaH80',
 'TravelfunRG',
 'swedenland',
 'djurmo1967',
 'Mailbutler',
 'Lenazb',
 'AlltomThailand',
 'annasverige10',
 'adamdansken',
 'sretna71',
 'dagenstraning',
 'sodper',
 'JonnySthlm',
 'LailaRomu',
 'kjaegerfalk',
 'MHidnas',
 'dabadoo_se',
 'nettan_jo',
 'paulinebdahl',
 'JoakimEriksson9',
 'sofiaalfredsson',
 'richardbarr2424',
 'Peabrainelin',
 'Colle_86',
 'ApPetersson']

We could, at this point, try to figure our which are the largest clusters.

In [31]:
clusBySize = sorted(userId.items(), key=lambda x: len(x[1]), reverse=True)

In [173]:
biggestClusters = clusBySize[0:20]
print ([clus[0] for clus in clusBySize[0:5]])

['3:1', '5:1', '6:1', '1:2', '1:1']


Build a corpus for tweets from users in the 20 largest clusters.

In [192]:
clusterTweets = []
for clu in biggestClusters:
    for user in userId[clu[0]]:
        if user in userTweets:
            string = ''
            for word in userTweets[user]: string += word
            clusterTweets.append(string)
print(len(clusterTweets))

1999


In [199]:
# from http://radimrehurek.com/gensim/tut1.html
stoplist = set('och i att för med så är det på som jag rt inte om the den men vi ni du en har av till de man ett kan var ska'.split())
texts = [[word for word in document.lower().split() if word not in stoplist]
          for document in clusterTweets]

In [200]:
print(texts[0][:5])

['@pemyh', 'prøver', 'å', 'være', 'grei']


In [201]:
# remove words that appear only once
from collections import defaultdict
frequency = defaultdict(int)
for text in texts:
        for token in text:
             frequency[token] += 1

texts = [[token for token in text if frequency[token] > 1] for text in texts]
# remove twitter handles
texts = [[token for token in text if token[0]!='@'] for text in texts]
# remove URLs (?)

In [203]:
print(texts[0][:5])

['prøver', 'å', 'være', 'grei', "ser'ru"]


Convert to vector space representation (or something like that)

In [205]:
dictionary = corpora.Dictionary(texts)
dictionary.save('clus_1_2.dict') # store the dictionary, for future reference
print(dictionary)

Dictionary(1654019 unique tokens: ['felsägning', 'alarm-bells', 'http://t.co/tckuoydcdw', 'http://t.co/x4gaj2c03r', 'gjeesp!']...)


AttributeError: 'Dictionary' object has no attribute 'popitem'

In [206]:
corpus = [dictionary.doc2bow(text) for text in texts]

In [184]:
import time
# Latent semantic indexing
start = time.time()
lsi = models.lsimodel.LsiModel(corpus=corpus, id2word=dictionary, num_topics=40)
elapsed = time.time() - start
print("Elapsed time", elapsed)

Elapsed time 166.2101309299469


In [185]:
for i in range(0,10):
    print(lsi.show_topic(i, topn=10))

[(0.5293467758035818, 'er'), (0.35538679526645728, 'jeg'), (0.31877828004754322, 'og'), (0.25474871589709769, 'at'), (0.25165526123770332, 'ikke'), (0.20650374334205807, 'for'), (0.20403147299768642, 'til'), (0.14435305909286361, 'å'), (0.11310836374981792, '-'), (0.10445684277767275, 'der')]
[(-0.21472843999407012, 'er'), (0.17310530158226667, 'när'), (0.17218650248011136, 'nu'), (-0.17019050208128611, 'jeg'), (0.16598077646049675, 'vad'), (0.1596662850624633, 'in'), (0.15411658243791093, 'ju'), (0.149686061049179, '#svpol'), (-0.14306047462720586, 'og'), (0.14111238447792435, '-')]
[(0.40012016385637289, 'to'), (0.28865357391691138, 'in'), (0.2837909946343114, 'of'), (0.2733140022400104, 'a'), (0.22265942268292099, 'on'), (0.21540865712649299, 'and'), (0.19259783456109972, 'is'), (0.18633877681562991, 'you'), (0.15539267775216745, 'for'), (-0.10557472976006232, ':)')]
[(0.84439099742407042, '#svpol'), (-0.15068187084983975, ':)'), (-0.11302436807726986, 'mig'), (0.10904972294269313, 

In [238]:
# Latent Dirichlet Allocation
start = time.time()
lda = models.ldamodel.LdaModel(corpus=corpus, id2word=dictionary, num_topics=10, update_every=1, chunksize=100, passes=100)
print("Elapsed time", time.time()-start, "s on ", len(corpus), " documents")

Elapsed time 166.2101309299469 s on  1999  documents


In [207]:
# TF-IDF model
tfidf = models.TfidfModel(corpus)


In [237]:
for clu in range(0,20):
    print("Cluster ", biggestClusters[clu][0], biggestClusters[clu][1][:5])
    print("=========")
    ixs = 100*clu
    ixe = 100*(clu+1)
    textInput = texts[ixs:ixe] 
    # Collapse to a single list
    textInputList = []
    for tweet in textInput:
        for word in tweet:
            textInputList.append(word)

    subset_tfidf = tfidf[dictionary.doc2bow(textInputList)]
    # Top terms
    topt = sorted(subset_tfidf, key=lambda x: x[1], reverse=True)[:10]
    # print(topt)
    for i in topt:
        print(dictionary[i[0]])
    print('\n')

Cluster  3:1 ['CSpange', 'Grest', 'Doremus42', 'mariesimonsen', 'Kvalshaug']
og
jeg
ikke
til
å
skal
meg
litt
hva
fra


Cluster  5:1 ['linusgulliksson', 'malinerson', 'linneaanonsen', 'wilmalindd', 'claudiakaninte']
tack
⠀
grattis
tackar
😘
jättemycket
jävla
mycket
😍
haha


Cluster  6:1 ['MxGustafsson', 'Dennnnis123', 'Erikinspace', 'mollerpontus', 'bokkkeen']
tack
⠀
grattis
😘
puss
finaste
❤️
jättemycket
hoppas
❤️❤️


Cluster  1:2 ['oisincantwell', 'Sakine', 'detljuvalivet', 'owenil', 'fvirtanen']
#svpol
#svtnyheter
#fofrk
regeringen
debatt:
#svt
sd
från
tack!
löfven


Cluster  1:1 ['GrayMunthe', 'GustaveLund', 'wistikent', 'kastrullis', 'mymlan']
ju
när
&gt;
:)
bara
&lt;3
vad
här
nåt
vara


Cluster  1:4 ['grishund', 'Anna_Kaya', 'braneback', 'jacobmollstam', 'HERO_Respondi']
#skolchatt
#linedu
#skolan
#afkrektor15
elever
#rektorschatt
lärare
eleverna
#afkumeå
#fskchatt


Cluster  1:3 ['TommyFunebo', 'perraponken', 'RolandXSweden', 'Stassministern', 'fogelklou']
#svpol
#migpol
islam
sd
#

[(73069, 0.252195563117452), (324058, 0.1341668783860213), (314439, 0.11167933303863017), (294201, 0.09246280637290397), (61249, 0.090559135336672)]


In [229]:
for i in topt:
    print(dictionary[i[0]])

#svpol
#svtnyheter
#fofrk
regeringen
debatt:
